In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable

import time
import argparse
import os
import sys
import numpy as np

# Our Modules
reader_folder = os.path.realpath(os.path.abspath('..'))
if reader_folder not in sys.path:
    sys.path.append(reader_folder)
import datasets
from datasets import utils
from models.MPNN import MPNN
from LogMetric import AverageMeter, Logger

reaction.py:48 <module> INFO Error importing py3Dmol


Error importing py3Dmol


In [2]:
root = os.path.join('../../../QMfiles/ts_xyz/')
files = [f for f in os.listdir(root) if os.path.isfile(os.path.join(root, f))]

In [37]:
files

['CC1=CCC2CC1C2(C)C+[H]_CC1=CC[C]2CC1C2(C)C+[H][H].xyz',
 'CC1=CCC2CC1C2(C)C+[CH3]_C+CC1=CCC2[CH]C1C2(C)C.xyz',
 'CO+[O]O_OO+[CH2]O.xyz',
 'CCCCCC=O+[O]O_CC[CH]CCC=O+OO.xyz',
 'CCCCCC(C)CO[O]+OO_CCCCCC(C)COO+[O]O.xyz',
 'OO+[CH2]C(=O)C=C(C)C_CC(=O)C=C(C)C+[O]O.xyz',
 'CCCC=CC(C)C+[CH3]_C+CC[CH]C=CC(C)C.xyz',
 'CCCC(CCC)O[O]+OO_CCCC(CCC)OO+[O]O.xyz',
 'CC1=CC=CO1+[O]O_OO+[CH2]C1=CC=CO1.xyz',
 'C[CH]C(=O)CCC+OO_CCCC(=O)CC+[O]O.xyz',
 'CCC(=O)COO+[O]O_C[CH]C(=O)COO+OO.xyz',
 'CC(C)[CH]O+OO_CC(C)CO+[O]O.xyz',
 'CCC(C)C+[O]O_OO+[CH2]C(C)CC.xyz',
 'CCCCC(C)CC+[CH3]_C+C[CH]CCC(C)CC.xyz',
 'CCCCCCCCC+[O]O_C[CH]CCCCCCC+OO.xyz',
 'CCCC(C)[CH]O+OO_CCCC(C)CO+[O]O.xyz',
 'C=CCCCC(C)C+[CH3]_C+C=[C]CCCC(C)C.xyz',
 '[CH2]C(C)CO+[O]O_OO+[CH2]C(C)[CH]O.xyz',
 'C=CCC[CH]CC+OO_C=CCCCCC+[O]O.xyz',
 'O=CO+[O]O_O=[C]O+OO.xyz',
 'CCCCCC(C)O[O]+OO_CCCCCC(C)OO+[O]O.xyz',
 'CCCCCCCCC=O+[O]O_CC[CH]CCCCCC=O+OO.xyz',
 'C=O+[O]O_OO+[CH]=O.xyz',
 'CC1=CC=CC(O)=C1+[O]O_OO+[CH2]C1=CC=CC(O)=C1.xyz',
 'CCCCC(C)CC+[H]_CC[

In [3]:
idx = np.random.permutation(len(files))
idx = idx.tolist()

In [4]:
len(idx)

948

In [5]:
test_ids = [files[i] for i in idx[:100]]

In [6]:
len(test_ids)

100

In [7]:
data_test = datasets.ts(root, test_ids)

In [8]:
# Define model and optimizer
print('Define model')
# Select one graph
g_tuple, l = data_test[0]
g, h_t, e = g_tuple

print('\tStatistics')
stat_dict = datasets.utils.get_graph_stats(data_test, ['degrees', 'target_mean', 'target_std', 'edge_labels'])


Define model
	Statistics


In [9]:
data_test.set_target_transform(lambda x: datasets.utils.normalize_data(x, stat_dict['target_mean'],
                                                                           stat_dict['target_std']))

In [10]:
test_loader = torch.utils.data.DataLoader(data_test,
                                              batch_size=20, collate_fn=datasets.utils.collate_g,
                                              num_workers=4, pin_memory=True)

In [11]:
model = MPNN([len(h_t[0]), len(list(e.values())[0])], 73, 15, 2, len(l), type='regression')

In [12]:
model_pth = torch.load('../checkpoint/ts/mpnn/model_best.pth')

In [13]:
model.load_state_dict(model_pth['state_dict'])

In [14]:
model.parameters

<bound method MPNN.parameters of MPNN(
  (m): ModuleList(
    (0): MessageFunction(
      (learn_args): ParameterList()
      (learn_modules): ModuleList(
        (0): NNet(
          (fcs): ModuleList(
            (0): Linear(in_features=5, out_features=128, bias=True)
            (1): Linear(in_features=128, out_features=256, bias=True)
            (2): Linear(in_features=256, out_features=128, bias=True)
            (3): Linear(in_features=128, out_features=1095, bias=True)
          )
        )
      )
    )
  )
  (u): ModuleList(
    (0): UpdateFunction(
      (learn_args): ParameterList()
      (learn_modules): ModuleList(
        (0): GRU(15, 73)
      )
    )
  )
  (r): ReadoutFunction(
    (learn_args): ParameterList()
    (learn_modules): ModuleList(
      (0): NNet(
        (fcs): ModuleList(
          (0): Linear(in_features=146, out_features=128, bias=True)
          (1): Linear(in_features=128, out_features=256, bias=True)
          (2): Linear(in_features=256, out_featur

In [15]:
model = model.cuda()

In [16]:
model = model.train()

In [17]:
output_list =[]
for i , (g,h,e,target) in enumerate(test_loader):
    g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
    g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)
    output = model(g, h, e)
    output_list.append(output)

In [22]:
print output_list

[tensor([[ 0.6485, -0.0764],
        [-0.8351,  0.4134],
        [-0.9376, -1.2426],
        [-0.3812,  1.6533],
        [-1.0691, -0.9594],
        [-0.7571, -1.3317],
        [ 0.8611,  0.1352],
        [ 0.3637,  0.1370],
        [ 0.6580, -0.1164],
        [ 0.1445,  0.0348],
        [-0.2917,  1.1819],
        [-0.6890,  1.8948],
        [ 0.5586, -0.2106],
        [ 0.3319,  0.2066],
        [-0.1391,  0.1728],
        [ 0.6430,  0.4816],
        [ 0.5997,  0.0850],
        [ 0.5190,  0.4489],
        [ 1.1571, -0.1325],
        [ 0.1974, -0.4368]], device='cuda:0'), tensor([[ 0.6104,  0.1514],
        [ 0.9832,  0.3047],
        [ 0.2701, -0.1782],
        [ 0.9946, -0.1236],
        [ 0.6765,  0.4834],
        [-0.9085, -0.8994],
        [ 0.7826,  0.0335],
        [-0.7538, -1.1264],
        [ 0.3078,  0.1858],
        [ 0.4312,  0.1765],
        [-0.3483,  1.9654],
        [-0.3733, -0.8690],
        [-0.0325,  0.4744],
        [ 0.3608, -0.4594],
        [ 0.3715,  0.0288],


In [27]:
batch_1 = output_list

In [29]:
batch_1 = np.array(batch_1.tolist())

In [33]:
(batch_1* stat_dict['target_std']) + stat_dict['target_mean']

array([[  88.55746496,   27.94111365],
       [  46.63070745,   29.26778024],
       [  43.73587293,   24.78309112],
       [  59.45901842,   32.62552528],
       [  40.0191355 ,   25.5498447 ],
       [  48.83441317,   24.54178846],
       [  94.56399603,   28.51422999],
       [  80.50851625,   28.51926443],
       [  88.82381197,   27.83284448],
       [  74.31521433,   28.2423888 ],
       [  61.9858688 ,   31.34895447],
       [  50.76061465,   33.27952524],
       [  86.01644654,   27.57766977],
       [  79.60885798,   28.70771808],
       [  66.30041697,   28.61599421],
       [  88.401043  ,   29.45227413],
       [  87.17696754,   28.37829168],
       [  84.89593091,   29.36385482],
       [ 102.92932919,   27.78919712],
       [  75.8073936 ,   26.96530137]])

In [34]:
import pandas as pd

In [35]:
param = pd.read_csv('../../../QMfiles/two_param.csv')

In [41]:
for i in idx[:100]:
    print param.ea2[i]

70.1091803997
77.6597645089
67.9961687539
40.6022695913
63.2924162212
65.7851216531
34.316590284
42.524291426
63.1949306835
70.5755447457
53.1820434298
105.655578814
70.6374307725
72.1307215791
45.2294375355
106.651491481
75.3622803299
35.3199428905
120.75088009
80.9281253218
83.3231475851
24.8720687717
87.5348123074
74.9320655324
95.0461178017
80.3977052922
28.3180050448
75.2079686444
111.388347399
190.134856127
32.0770148653
106.49305019
90.8060135407
36.2677634469
58.9641790556
83.9479606278
88.9504363333
75.1716918732
36.5939180594
62.6562755161
92.9560462213
93.6283817496
80.8426198727
78.6100595006
75.4921832836
64.4870360073
69.0491053954
73.2620872411
39.6863225517
56.5606277714
64.9771193731
96.3904984969
50.4883660676
65.3924365871
101.70604547
98.6248359837
124.157339192
57.2359183342
37.6759984447
57.1220003214
40.5603432993
33.3798091724
83.7572138138
41.7848344413
97.4483541883
55.499939311
55.3336476101
114.890944546
68.0777830304
87.9577739049
26.3102946254
47.298398672

In [47]:
array_list = []
for batch in output_list:
    array_list.append(batch.tolist())

In [48]:
array_list = np.array(array_list)
array_list = (array_list* stat_dict['target_std']) + stat_dict['target_mean']

In [52]:
array_list

array([[[  88.55746496,   27.94111365],
        [  46.63070745,   29.26778024],
        [  43.73587293,   24.78309112],
        [  59.45901842,   32.62552528],
        [  40.0191355 ,   25.5498447 ],
        [  48.83441317,   24.54178846],
        [  94.56399603,   28.51422999],
        [  80.50851625,   28.51926443],
        [  88.82381197,   27.83284448],
        [  74.31521433,   28.2423888 ],
        [  61.9858688 ,   31.34895447],
        [  50.76061465,   33.27952524],
        [  86.01644654,   27.57766977],
        [  79.60885798,   28.70771808],
        [  66.30041697,   28.61599421],
        [  88.401043  ,   29.45227413],
        [  87.17696754,   28.37829168],
        [  84.89593091,   29.36385482],
        [ 102.92932919,   27.78919712],
        [  75.8073936 ,   26.96530137]],

       [[  87.47867948,   28.55804131],
        [  98.0157929 ,   28.97329443],
        [  77.86195551,   27.66548554],
        [  98.3373652 ,   27.81338405],
        [  89.34828628,   29.45730796]